In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Reshape, Lambda, Dropout, LSTM, Embedding, Activation
from tensorflow.keras.optimizers import Adam
import emoji
import csv
import pandas as pd
from demo_utils import *


In [ ]:
X_train, Y_train = pd.read_csv("data/train_emoji.csv")
X_test, Y_test = pd.read_csv("data/tesss.csv")
Tx = len(max(X_train, key = len).split())

In [ ]:
train_process = normalize_text(train_set.copy()) 
test_process = normalize_text(test_set.copy())

In [ ]:
X_train = train_process.text.to_numpy()
X_test = test_process.text.to_numpy()
Y_train = train_process.label.to_numpy()
Y_test = test_process.label.to_numpy()

In [ ]:
Y_train_oh = one_hot_y(Y_train)
Y_test_oh = one_hot_y(Y_test)

word_to_index_oh, index_to_word_oh = get_word_beg(X_train)
X_train_indices = x_to_indices(X_train, Tx, word_to_index_oh)
X_test_indices = x_to_indices(X_test, Tx, word_to_index_oh)
X_train_oh = one_hot_x(X_train_indices, word_to_index_oh)
X_test_oh = one_hot_x(X_test_indices, word_to_index_oh)

In [ ]:
word_size = X_train_oh.shape[2]
n_a = 64

In [ ]:
def model_v1(input_shape, n_a):
    
    sentence_indices = Input(shape = input_shape, dtype = np.float64)
    
    X = LSTM(units = n_a, return_sequences = True)(sentence_indices)
    X = Dropout(0.5)(X)
    X = LSTM(units = n_a, return_sequences = False)(X)
    X = Dropout(0.5)(X)
    X = Dense(5, activation = "softmax")(X)
    
    model = Model(inputs = sentence_indices, outputs = X)
    
    return model

In [ ]:
modelV1 = model_v1((Tx, word_size), n_a)

In [ ]:
modelV1.summary()

In [ ]:
opt = Adam(learning_rate = 0.01, beta_1 = 0.9, beta_2 = 0.999, decay = 0.01)
modelV1.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
history = modelV1.fit(X_train_oh, Y_train_oh, epochs = 10, batch_size = 32)

In [ ]:
loss, acc = modelV1.evaluate(X_test_oh, Y_test_oh)